### Group 28 members:
- Jingze Tian (CCID)
- Letian Ren (CCID)
- Essam Gouda (egouda)

# Task 2: Classification
- Logistic Regression model for female OR male mosquito = function(weather_features)
- Support Vector Machine model for female OR male mosquito = function(weather_features)
- Random Forest model for female OR male mosquito = function(weather_features)

#### Variations for each model:
- 10-fold cross validation for each model was performed
- t-test and ANOVA were used to compare model's performance